# Ускорение Python

## Первоначальные данные

In [ ]:
import random

In [ ]:
class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

In [ ]:
def matrix_product(X, Y):
    """Computes the matrix product of X and Y.

    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6]])
    >>> matrix_product(X, Y)
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >>> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # верим, что с размерностями всё хорошо
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]

    return Z

In [ ]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [ ]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])

In [ ]:
matrix_product(X, Y)

[[4, 5, 6], [8, 10, 12], [12, 15, 18]]

In [ ]:
matrix_product(Y, X)

[[32]]

In [ ]:
%doctest_mode

Exception reporting mode: Context
Doctest mode is: OFF


# Измерение времени работы

Кажется, всё работает, но насколько быстро? Воспользуемся "магической" командой `timeit`, чтобы проверить.

In [ ]:
import timeit

In [ ]:
setup = """
import random
from app import Matrix, matrix_product
shape = 64, 64
X = Matrix.rnd(shape)
Y = Matrix.random(shape)
"""

In [ ]:
# timeit.timeit("matrix_product(X, Y)", setup, number=10)

In [ ]:
%%timeit shape = 64, 64; X = Matrix.random(shape); Y = Matrix.random(shape)
matrix_product(X, Y)

83.8 ms ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Y U SO SLOW?

Определим вспомогательную функцию `bench`, которая генерирует случайные матрицы указанного размера, а затем `n_iter` раз умножает их в цикле.

In [ ]:
def bench(shape=(64, 64), n_iter=16):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for iter in range(n_iter):
        matrix_product(X, Y)

Воспользуемся модулем `cProfile` для поиска проблемы.

In [ ]:
import cProfile

In [ ]:
source = open("faster_python.py").read()
cProfile.run(source, sort="tottime")

         65957 function calls in 2.081 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    2.030    0.127    2.031    0.127 <string>:25(matrix_product)
     8192    0.019    0.000    0.033    0.000 random.py:292(randrange)
      128    0.008    0.000    0.047    0.000 <string>:11(<listcomp>)
     8192    0.008    0.000    0.011    0.000 random.py:239(_randbelow_with_getrandbits)
     8192    0.006    0.000    0.039    0.000 random.py:366(randint)
    24576    0.003    0.000    0.003    0.000 {built-in method _operator.index}
     8205    0.002    0.000    0.002    0.000 {method 'getrandbits' of '_random.Random' objects}
     8192    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
        1    0.001    0.001    2.080    2.080 <string>:46(bench)
        1    0.001    0.001    2.081    2.081 {built-in method builtins.exec}
       16    0.000    0.000    0.000    0.000 <string>:5(<listcomp>)
   

Результат предсказуемый и довольно бесполезный: >90% времени работы происходит в функции `matrix_product`. Попробуем посмотреть на неё по внимательней с помощью модуля `line_profiler`.

In [ ]:
!pip install line_profiler

In [ ]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
%lprun -f matrix_product bench()

Заметим, что операция `list.__getitem__` не бесплатна. Переставим местами циклы `for` так, чтобы код делал меньше обращений по индексу.

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [ ]:
%lprun -f matrix_product bench()

Немного быстрее, но всё равно слишком медленно: >30% времени уходит исключительно на итерацию! Поправим это.

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

In [ ]:
%lprun -f matrix_product bench()

Функции `matrix_product` сильно похорошело. Но, кажется, это не предел. Попробуем снова убрать лишние обращения по индексу из самого внутреннего цикла.

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # <--
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

# Numba

Numba не работает с встроенными списками. Перепишем функцию `matrix_product` с использованием ndarray.

In [ ]:
import numba
import numpy as np


@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

Посмотрим, что получилось.

In [ ]:
shape = 64, 64
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

%timeit -n100 jit_matrix_product(X, Y)

The slowest run took 18.98 times longer than the fastest. This could mean that an intermediate result is being cached.
626 µs ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Cython

In [ ]:
!pip install cython

In [ ]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [ ]:
%%cython -a
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

<IPython.core.display.HTML object>

In [ ]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [ ]:
%timeit -n100 cy_matrix_product(X, Y)

24.1 ms ± 3.99 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Проблема в том, что Cython не может эффективно оптимизировать работу со списками, которые могут содержать элементы различных типов, поэтому перепишем `matrix_product` с использованием *ndarray*.

In [ ]:
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

In [ ]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

<IPython.core.display.HTML object>

In [ ]:
%timeit -n100 cy_matrix_product(X, Y)

158 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Как же так! Стало только хуже, причём большинство кода всё ещё использует вызовы Python. Избавимся от них, проаннотировав код типами.

In [ ]:
%%cython -a
import numpy as np
cimport numpy as cnp

def cy_matrix_product(cnp.ndarray X, cnp.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef cnp.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

<IPython.core.display.HTML object>

In [ ]:
%timeit -n100 cy_matrix_product(X, Y)

165 ms ± 7.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


К сожалению, типовые аннотации не изменили время работы, потому что тело самого вложенного цикла Cython оптимизировать не смог. Fatality-time: укажем тип элементов в *ndarray*.

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

<IPython.core.display.HTML object>

In [ ]:
%timeit -n100 cy_matrix_product(X, Y)

441 µs ± 9.92 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Попробуем пойти дальше и отключить проверки на выход за границы массива и переполнение целочисленных типов.

In [ ]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

<IPython.core.display.HTML object>

In [ ]:
%timeit -n100 cy_matrix_product(X, Y)

263 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
